# Условие

Дана выборка линейно разделимая выборка $X=\{x_1,,x_2,\dots,x_n\}:\ x_i\in \mathbb{R}^m$, $Y=\{y_1,y_2,\dots,y_n\}: y_i=\pm 1$.
<br>
Необходимо найти разделяющую гиперплоскость, проходящую через начало координат, т. е. найти такой вектор $w$:
$$sign\left(x_iw\right)=sign\left(\sum_{j=1}^mx_{ij}w_j\right)=y_i,\forall i$$
где $x_i$ - вектор-строка, $w$ - вектор-столбец, $y_i$ - число.

# Решение

**Логистическая регрессия**
<br>
Просто напишем логистическую регрессию, выведем в ответ компоненты гиперплоскости. Используем батчи для ускорения. Вместо количества эпох условием выхода из цикла будет проверка того, что каждый элемент выборки оказался со своей стороны от разделяющей гиперплоскости. Это нестабильно в случае, если выборка не линейно разделима, но это не наш случай.

**Персептрон**
<br>
Для нахождения разделяющей поверхности проще использовать алгоритм персептрона (Novikoff, 1962). Это неградиентный алгоритм, который не работает, если выборка линейно не разделима. Но если выборка линейно разделима, проще использовать этот алгоритм, так как его и написать гораздо проще, и настраивать не нужно гиперпараметры.
<br>
Этот алгоритм идеально подходит под олимпиадные или учебные задачи, но в реальности, где выборка почти никогда не может быть идеально разделена даже нелинейно, используют градиентные способы аппроксимации фукнции ошибки. Это позволяет в общем улучшать модель, плавно уменьшая ошибку, в то время как ошибка перцептрона скачет туда-сюда, и планомерно уменьшить её не получится - либо ошибка 0, либо непонятно какая.
<br>
Общая мысль алгоритма такова. Пусть имеется вектор $w$, при этом не для всех выполняется условие $y_i * (x_iw)>0$ ($y_i$ - число, остальное векторы).
<br>
Тогда, возьмём один из таких $x_i$, для которых условие не выполняется и прибавим $y_i * x_i$ к $w:=w+y_i * x_i$, то есть перетянем $w$ в сторону $x_i$ (если $y_i=1$, иначе - в противоположную сторону). После этого угол между $w$ и $x_i$ уменьшится (уменьшится между $w$ и $-x_i$ при $y_i=-1$), а значит приблизится ситуация, когда угол между ними станет $<90$. А если угол меньше 90, то и скалярное произведение (это наш случай - $x_iw$) будет больше нуля, так как скалярное произведение можно выразить через косинус, который больше 0 при угле меньше 90.
<br>
Novikoff доказал теорему о том, что если так постоянно перетягивать вектор $w$ к тем $x_i$, на которых он ошибается, через конечное число итераций алгоритм сойдётся.

# Код

**Логистическая регрессия**

In [25]:
import numpy as np

gen_rand_rng = np.random.default_rng(seed=42)

def gen_batches(X, Y, batch_size):
    perm = gen_rand_rng.permutation(X.shape[0])
    X_perm, Y_perm = X[perm], Y[perm]
    for i in range(0, X.shape[0], batch_size):
        yield X_perm[i:min(i + batch_size, X.shape[0])], Y_perm[i:min(i + batch_size, X.shape[0])]

class LogisticRegression:
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.W = gen_rand_rng.standard_normal((X.shape[1], Y.shape[1]))
        self.batch_size = max(round(self.X.shape[0]**0.5), 100) # batches_size = sqrt(N)

    def _isDivided(self):
        return np.all(np.sign(self.X @ self.W) == np.sign(self.Y-0.5))

    def _predict(self, X):
        return 1 / (1 + np.exp(-X @ self.W))

    def fit(self):
        while not self._isDivided():
            for X_batch, Y_batch in gen_batches(self.X, self.Y, self.batch_size):
                Y_pred = self._predict(X_batch)
                grad = (X_batch.T @ (Y_pred - Y_batch)) / X_batch.shape[0]
                self.W -= 0.01 * grad
                # Важно не проверять _isDivided() внутри цикла for, потому что такая проверка работает за O(N), тогда если батчей sqrt(N),
                # одна эпоха вместо O(N) действий выполняет O(N^1.5), что дольше, чем просто считать градиент по всей выборки без батчей
            
n, m = [int(x) for x in input().split()]
X, Y = [], []
for i in range(n):
    data = [float(x) for x in input().split()]
    X.append(data[:-1])
    Y.append(1 if data[-1] == 1 else 0)
X = np.array(X)
Y = np.array(Y).reshape((-1, 1))
model = LogisticRegression(X, Y)
model.fit()
print(*model.W.ravel())

 2 1
 -1 -1
 1 1


0.30896112736195985


**Персептрон**

In [6]:
import numpy as np

n, m = [int(x) for x in input().split()]
X, Y = [], []
for i in range(n):
    data = [float(x) for x in input().split()]
    X.append(data[:-1])
    Y.append(data[-1])
X = np.array(X)
Y = np.array(Y)
W = np.zeros(m)
error_appeared = True
while error_appeared:
    error_appeared = False
    for i in range(n):
        if X[i] @ W * Y[i] <= 0:
            error_appeared = True
            W += X[i].T * Y[i]
print(*W)

 2 2
 0 1 1
 1 1 -1


-2.0 1.0
